# How to create a submission in codabench

The aim of this notebook is to show how to create a submission. It present necessary and optionnal components.

#### Submission samples are available in the `submission` folder

Those examples shows how to configure a submission for the following use cases :
- Simple submission using pre-implemented functions
- Submission with a custom model
- Submission with a custom scaler
- Submission for a pre-trained model
- Submission for scoring only

Note : in the final phase of the competition, submission will always be re-trained to insure fairness.
Note 2 : implementation of tensorflow is being prepared, currently the submission process only accept pytorch models.

**This Notebook present the composition of a submission folder, for examples see notebook 7 :  `submission_examples` and related folder `submission`, for information on the technical environment see notebook 8  `technical environment`**

## Composition

A submission is will need to have the following files (depending on the submission type, see example):
- parameters.json (mandatory) : file containing all the necessary parameters to configure a submission
- config.ini : configuration file for the augmented simulator (which implement the model)
- scaler_parameters.py : file containing a function that return the parameters for the scaler
- my_augmented_simulator.py : file containing an implementation of a custom augmented simulator (which implement the model) that will be evaluated
- my_scaler.py : file containing the implementation of a custom scaler
- results.json : file containing pre-calculated results for which only the scores are evaluated (see notebook 4 and 5 for an example of implementation and notebook 7 for an example of file)

There is also the possibility to add a folder for a pre-trained model:
- trained_model : will contains the pre-trained model saved through an implementation of the "save" function (see notebook 7 for an example)

## Submission
Submission in codabench will be made through the "My Submissions" tab. A zip containing the files associated with the submission is uploaded.
**Be careful to only compress the file and not the folder containing the file**

## Files content

### parameters.json

This file is used to describe the submission. It indicates to the server which type of submission is being run (training, without training, scoring only) and how to configure it. 

In [ ]:
{
  "evaluateonly": false,
  "scoringonly": false,
  "simulator_config": {
    "simulator_type": "simple_torch",
    "simulator_file": "my_augmented_simulator",
    "name": "MyAugmentedSimulator",
    "model": "TorchFullyConnected",
    "model_type": "fully_connected",
    "scaler_type": "None",
    "scaler_file": "None",
    "scaler_class": "standard_scaler",
    "scaler": "StandardScaler",
    "config_name": "DEFAULT",
  },
  "simulator_extra_parameters": {},
  "training_config": {
    "epochs": 1
  }
}


- `evaluateonly` : set to true if training was already done and the trained model is provided in the `trained_model` folder. 
- `scoringonly` : set to true if training and evaluation was already done and a `results.json` file is provided.
##### simulator_config
This object is used to indicate the configuration of the augmented simulator :
- `simulator_type`: string indicates the type of model being provided : `simple_torch`, `custom_torch` (LIPS model provided in simulator_file), `simple_tf`, `custom` (model provided in simulator_file)
- `simulator_file`: File in which the custom model can be found (at the root of the folder) **without the '.py'**
- `name`: Name of the simulator
- `model`: Model name, either of the implementation in `simulator_file.py`or of an available model in LIPS
- `model_type`: Used for model already implemented in LIPS for loading  
- `scaler_type`: string indicates the type of model being provided : `simple`, `custom`, `None` (note, for a custom simulator implementation the scaler need to be implemented in the model, no scaler is passed)
- `scaler_file`: File in which the custom scaler can be found (at the root of the folder) **without the '.py'**
- `scaler_class`: Used for scaler already implemented in LIPS for loading
- `scaler`: Name of the scaler to be loaded either from LIPS or `scaler_file`
- `config_name`: Name of the configuration to be used from `config.ini`
##### simulator_extra_parameters
- `simulator_extra_parameters`: An object containing custom input parameters for the model, it will be passed to the model. 
##### training_config
- `training_config`: An object containing input parameters for the training phase, it will be passed when the `train` function is called. 


### config.ini

Configuration file for the model, see `4_How_to_contribute`, the section defined by `config_name` will be used.

### scaler_parameter.py

This file is used to generate the parameters for the scaler. The example provided show how to use it for recreating the 1st example from `4_How_to_contribute`

### my_augmented_simulator.py

This file is used to implement a custom model. This model needs to be runnable by the ingestion process. Notebook 7 presents example for reproducing the 2nd example from `4_How_to_contribute` and `3b_Reproduce_baseline_result_advanced_configuration`. 
A custom model will need to implement the following function in order to be runnable in the ingestion process :
- __init__(self,benchmark,**kwargs)
- train(self,train_dataset, save_path=None)
- predict(self,dataset,**kwargs)

### my_scaler.py

This file is used to implement a custom scaler. The example provided show how to use it for recreating an already implemented scaler (used in `4_How_to_contribute`). 

### results.json

Used in combination with `scoringonly`: true, no training or evaluation will be run, the file will be send directly for scoring (only available during the warmup phase)

### trained_model

Used in combination with `evaluateonly`: true, contains a saved trained model (using the save function from the simulator). In this case, training is skipped and the saved model loaded.